# Simple Index Demo

#### Load documents, build the GPTSimpleVectorIndex

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [3]:
index = GPTSimpleVectorIndex.from_documents(documents)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17617 tokens
> [build_index_from_nodes] Total embedding token usage: 17617 tokens


In [6]:
# save index to disk
index.save_to_disk('index_simple.json')

In [4]:
# load index from disk
index = GPTSimpleVectorIndex.load_from_disk('index_simple.json')

#### Query Index

In [8]:
# set Logging to DEBUG for more detailed outputs
response = index.query("What did the author do growing up?")

INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 4154 tokens
> [query] Total LLM token usage: 4154 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 8 tokens
> [query] Total embedding token usage: 8 tokens


In [9]:
display(Markdown(f"<b>{response}</b>"))

<b>

The author grew up writing short stories, programming on an IBM 1401, and building a computer kit with a friend. He also wrote programs for a TRS-80 computer, such as games, a program to predict model rocket flight, and a word processor. He studied philosophy in college, but switched to AI and taught himself Lisp. He wrote a book about Lisp hacking and reverse-engineered SHRDLU for his undergraduate thesis. He visited Rich Draves at CMU and realized he could make art that would last. He took art classes at Harvard and applied to RISD and the Accademia di Belli Arti in Florence. He passed the entrance exam in Florence and studied painting at the Accademia, arriving at an arrangement whereby the students wouldn't require the faculty to teach anything, and in return the faculty wouldn't require the students to learn anything. He also painted still lives in his bedroom at night, using leftover scraps of canvas.</b>

**Query Index with SVM/Linear Regression**

Use Karpathy's [SVM-based](https://twitter.com/karpathy/status/1647025230546886658?s=20) approach. Set query as positive example, all other datapoints as negative examples, and then fit a hyperplane.

In [ ]:
query_mode = "svm"
# query_mode = "linear_regression"
# query_mode = "logistic_regression"

# set Logging to DEBUG for more detailed outputs
response = index.query(
    "What did the author do growing up?", vector_store_query_mode=query_mode
)

In [9]:
display(Markdown(f"<b>{response}</b>"))

<b>

The author grew up writing short stories, programming on an IBM 1401, and building a computer kit with a friend. They also wrote programs for a TRS-80 computer, such as games, a program to predict model rocket flight, and a word processor. In college, they studied philosophy and AI, and wrote a book about Lisp hacking. They also took art classes and applied to art schools, and while a student at the Accademia, they started painting still lives in their bedroom at night. These paintings were tiny, because the room was, and because they painted them on leftover scraps of canvas, which was all they could afford at the time. They also arrived at an arrangement with the faculty whereby the students wouldn't require the faculty to teach anything, and in return the faculty wouldn't require the students to learn anything. They even had a little stove, fed with kindling, that you see in 19th century studio paintings, and a nude model sitting as close to it as possible without getting burned. The author also painted the model, while the other students spent their time chatting or occasionally trying to imitate things they'd seen in American art magazines. The model turned out to live just down the street from the author, and made a living from a</b>

In [10]:
print(response.source_nodes[0].source_text)

		

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in

/Users/jerryliu/Programming/gpt_index/gpt_index/data_structs/node_v2.py:176: UserWarning: .source_text is deprecated, use .node.get_text() instead
  warnings.warn(".source_text is deprecated, use .node.get_text() instead")


**Query Index with custom embedding string**

In [ ]:
from gpt_index.indices.query.schema import QueryBundle

In [ ]:
query_bundle = QueryBundle(
    query_str="What did the author do growing up?", 
    custom_embedding_strs=['The author grew up painting.']
)
response = index.query(query_bundle)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

#### Get Sources

In [ ]:
print(response.get_formatted_sources())

#### Query Index with LlamaLogger

Log intermediate outputs and view/use them.

In [6]:
from gpt_index.logger import LlamaLogger
from gpt_index import ServiceContext

llama_logger = LlamaLogger()
service_context = ServiceContext.from_defaults(llama_logger=llama_logger)

In [ ]:
response = index.query(
    "What did the author do growing up?",
    service_context=service_context,
    similarity_top_k=2,
    # response_mode="tree_summarize"
)

In [ ]:
# get logs
service_context.llama_logger.get_logs()